# Introduction

The aim of this project is to create a retrieval based ChatBot using predefined patterns and responses.

Table of contents:<br>
* [1.Import and Load](#importandload)
* [2.Preprocess](#preprocess)
* [3.Train and Test Data](#traintest)
* [4.RNN](#model)
* [5.Graphical User Interface and Prediction](#gui)

###### Team members: Name (Registration Number):
###### Vakouftsis_Athanasios  (2022202004002), Tsarouchi Alexandra(2022202004024)

### 1.Import and Load
<a id="importandload"></a> 

In [ ]:
#The data we used are saved in intents.json file and contains intents like the one below
    {"tag": "greeting",
         "patterns": ["Hi there", "How are you", "Is anyone there?","Hey","Hola", "Hello", "Good day"],
         "responses": ["Hello, what do you want to know?", "Good to see you again, what would you like to know?", "Hi there, how can I help?"],
         "context": [""]
    }
#Where tag describes the type of intent, patterns the type of question the user want to ask and responses describes the one
#random response the bot will give. So we import the json file with:

intents = json.loads(open('intents.json').read())

### 2.Preprocess
<a id="preprocess"></a>

In [ ]:
#As for the preprocess of the data we tokenize each word and add it to a list of words and then we add each word in a document
#with the tag that this word has and we add each class on a list.Below we can see the code:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #Tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #Add documents in the corpus
        documents.append((w, intent['tag']))

        #Add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

#After, we lemmatize every word with lemmatizer and we save the words and the classes on pickle files respectively.
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

### 3.Train and Test Data
<a id="traintest"></a> 

In [ ]:
#We create our training data with the code below:
for doc in documents:
    #Initialize our bag of words
    bag = []
    #List of tokenized words for the pattern
    pattern_words = doc[0]
    #Lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    #Create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    #Output is a '0' for each tag and '1' for each pattern
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

#Then we create two lists train_x(containing all patterns with all the words) and train_y(containing all patterns with all the intents)
#and we divide the 90% of the data to x_train, y_train and 10% of the data to x_test,y_test for our model and the evaluation.

### 4. RNN
<a id="model"></a> 

In [ ]:
#We used a simple RNN and not an LSTM because we had very good accuracy(around 96% and on some occasions up to 100%). We found
#that with the baseline of a RNN(one input layer,one hidden layer and one output layer) didn't get the desirable results so
#we used 3 layers and 2 hidden layers. We found that 80 neurons for the input layer was the sweet spot between accuracy and not
#overfitting the model. The output layer has the length of the y_train which is the number of classes used for training. Finally
#we used SGD optimizer and nesterov for the best possible results. Below we can see the mdoel:
model = Sequential()
model.add(Dense(80, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(40, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

#Compile model with Stochastic gradient descent with Nesterov the best possible results
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

### 5.Graphical User Interface and Prediction
<a id="gui"></a> 

In [ ]:
#Below we can see the function in which we predict the class of the users response:
def predict_class(sentence, model):
    #Filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    #Sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

#Then we created an interface in order to communicate with the ChatBot. We used tkinter library to create the interface and the
#user can either press the send button to send the message or press the ENTER key of the keyboard.
#For more information there is all the code in the python files, we used PyCharm to create the ChatBot.